# Introduction
- This introduction is the easiest way for Python folks to play with Percolate
- We provider the Postgres client using the Docker compose connection parameteres as defaults
- We can learn how to register entities after which everyone else becomes possible

In [ ]:
import requests
#requests.get('http://www.percolationlabs.ai').content

In [ ]:
import percolate
from percolate.services import PostgresService
pg = PostgresService()

## Registering entities which are Agents is the key way to use Percolate
- entity models describe agents completely
- Percolate assumes agents are serializable in the "Agent" model which has structured response, sytem prompt and external function refs
- We register Entities as tables that can store data, as Agents that can allow interaction with data
- Agents are added as structured tables but other indexes are added too; embeddings on fields that need them and graph node registry

In [ ]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy

#we register the core models - these are added by scripts in install anyway but it illustrates for 'bring your own'
for model in [Agent]:
    repo = pg.repository(model)
    repo.register(register_entities=True)

## Percolate configures Langauge models and assumes we can load tokens from the env
- While not recommended in production, for simplicity you might add tokens locally in your database
- The example below registers unique model names - the name is sometimes of the from provider-model if you want to select an API

In [1]:
"""these are stored in the database by default and you can add your own
in the python client your API token is used
in the database you can save the token but the extension can also load it from the environment if its maintained on the database cluster
"""
from percolate.models.p8 import sample_models

## Adding APIs
- Percolate registers apis and functions 
- for example a freely available test api is at https://petstore.swagger.io/#/pet/findPetsByStatus
- we add this to percolate below

In [1]:
from percolate.utils.ingestion import add 
import os

# add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

In [2]:
import percolate as p8
from percolate.models.p8 import Function
"""filter by the proxy, verb etc if needed"""
functions = [Function(**f) for f in p8.repository(Function).select()]
"""functions are callable"""

'functions are callable'

#### any database loaded function can be called and you can use the metadata to know how to call it(as can an llm)

### the function manager is responsible for finding functions in the database but you can also ref external functions on your agent
under the hood if any agent either asks to load a function or if the function is added as a reference in it's `get_functions` the funciton manager is used to load the function into context
```python
from percolate.services import FunctionManager
f = FunctionManager()
f.add_functions_by_key('get_pet_findByStatus')
```

### Example of how we bootsrap the database by adding the P8 models. 

In [ ]:
def bootstrap(root='../../../extension/sql'):
    """util to generate the sql that we use to setup percolate"""
    
    from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy, PlanModel
    from percolate.models.p8 import sample_models
    from percolate.models.utils import SqlModelHelper
    import glob

    root = root.rstrip('/')
    
    models = [ Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy, PlanModel]
        
    """compile the functions into one file"""
    with open(f'{root}/01_add_functions.sql', 'w') as f:
        for sql in glob.glob('../../../extension/sql-staging/p8_pg_functions/**/*.sql',recursive=True):
            with open(sql, 'r') as sql:
                f.write(sql.read())
                f.write('\n\n---------\n\n')

    """add base tables"""            
    with open(f'{root}/02_create_primary.sql', 'w') as f:
        for model in models:
            f.write(pg.repository(model).model_registration_script(secondary=False, primary=True))

    """add the rest"""
    with open(f'{root}/03_create_secondary.sql', 'w') as f:    
        for model in models:
            f.write(pg.repository(model).model_registration_script(secondary=True, primary=False))
        script = SqlModelHelper(LanguageModelApi).get_data_load_statement(sample_models)
        f.write('\n\n-- -----------\n')
        f.write('-- sample models--\n\n')
        
        f.write(script)
bootstrap(root='../../../extension/sql')